# IoT Project

##### To get started

We need the following Python libraries:  
- Scapy: For parsing network packet traces
- IPy: For each IP addressing types (what is a local network address vs what is public network address, etcc)
- manuf: Look up a device manufacturer based on MAC address data

In [51]:
!pip install scapy IPy manuf pycurl StringIO

  Could not find a version that satisfies the requirement StringIO (from versions: )
No matching distribution found for StringIO


In [1]:
from scapy.all import *
from manuf import manuf

In [11]:
#help(scapy.all)

In [2]:
packets =rdpcap("./packet_capture.pcap")
p = manuf.MacParser(update=True)
# Example for testing Manufacturer look up based on MAC ID
p.get_all('84:38:35:5a:70:40')[0]

'Apple'

In [6]:
import numpy as np
import pandas as pd
from collections import defaultdict
from scapy.all import *
from IPy import IP as IPy
import pprint

d= defaultdict(list)

for i in range(len(packets)):
#for i in range(500):
    packet = packets[i]
    ## traffic patterns:
    # ARP: ignore for now
    # let's not worry about IPv6 for now
    if (packet.haslayer(IPv6)):
        continue
    # DNS
    if(packet.haslayer(DNS)):
        # look for DNS responses 
        #print(packet.summary())
        #is it DNS or mDNS traffic
        if (packet.haslayer(ICMP)):
            # this is a DNS request within ICMP
            mac_id = packet["Ether.src"]
            IP_src = packet["IP.src"]
            dns_qd_qname = str(packet[DNS].qd.qname)
            if mac_id not in d:
                d[mac_id] = {
                            'Manufacturer': p.get_all(mac_id)[0],
                            'DNS QD': dns_qd_qname
                            }
            else:
                d[mac_id]['DNS QD'] = dns_qd_qname
        elif (packet[UDP].sport == 5353):
            # this is mDNS
            mac_id = packet["Ether.src"]
            IP_src = packet["IP.src"]
            if (packet[DNS].an != None):
                mdns_an_rrname = str(packet[DNS].an.rrname)
                if mac_id not in d:
                    d[mac_id] = {
                                'Manufacturer': p.get_all(mac_id)[0],
                                'MDNS AN': mdns_an_rrname
                                }
                else:
                    d[mac_id]['MDNS AN'] = mdns_an_rrname
            elif (packet[DNS].ns != None):
                mdns_ns_rrname = str(packet[DNS].ns.rrname)
                if mac_id not in d:
                    d[mac_id] = {
                                'Manufacturer': p.get_all(mac_id)[0],
                                'MDNS NS': mdns_ns_rrname
                                }
                else:
                    d[mac_id]['MDNS NS'] = mdns_ns_rrname
        #elif (packet[UDP].sport == 53):
        else:
            # this is DNS
            # these packets come from the DNS server to the IoT device
            # so we are interested in the destination mac ID
            mac_id = packet["Ether.dst"]
            IP_src = packet["IP.src"]
            IP_dst = packet["IP.dst"]
            if (packet[DNS].qd != None):
                dns_qd_qname = str(packet[DNS].qd.qname)
                if mac_id not in d:
                    d[mac_id] = {
                                'Manufacturer': p.get_all(mac_id)[0],
                                'DNS QD': dns_qd_qname
                                }
                else:
                    d[mac_id]['DNS QD'] = dns_qd_qname
    # HTTP:
    elif(packet.haslayer(IP)):
        if (packet.haslayer(TCP)):
            # This is a terrible hack for detecting http traffic
            if (packet[TCP].dport == 80):
                mac_id = packet["Ether.src"]
                IP_src = packet["IP.src"]
                IP_dst = packet["IP.dst"]
                # is there a real payload?
                if(len(packet[TCP].payload) > 10):
                    payload = str(packet[TCP].payload)
                else:
                    payload = None
                if (IPy(IP_src).iptype() == "PRIVATE"):
                    if (IPy(IP_dst).iptype() == "PUBLIC"):
                        if mac_id not in d:
                            d[mac_id] = {
                                        'Manufacturer': p.get_all(mac_id)[0],
                                        'HTTP': {IP_dst: payload}
                                        }
                        else:
                            d[mac_id]['HTTP'][IP_dst] = payload  
    # Else: What did I miss
    else:
        print(packet.summary())
    
    
    # if the device is in the dataframe
    #df.loc[i] = [mac_id, p.get_all(mac_id)[0]]
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(d)

Ether / Dot1Q / ARP who has 192.168.1.217 says 192.168.1.1
Ether / Dot1Q / ARP who has 192.168.1.232 says 192.168.1.1
Ether / Dot1Q / ARP who has 192.168.1.14 says 192.168.1.1
Ether / Dot1Q / ARP is at 88:71:e5:d2:73:4b says 192.168.1.217 / Padding
Ether / Dot1Q / ARP is at 78:28:ca:03:80:0c says 192.168.1.232 / Padding
Ether / Dot1Q / ARP is at 84:38:35:5a:70:40 says 192.168.1.14 / Padding
Ether / 802.1q 00:18:0a:7d:01:cf > 01:80:c2:00:00:00 (0x27) vlan 1 / LLC / STP / Raw / Padding
Ether / Dot1Q / ARP who has 192.168.1.235 says 192.168.1.1
Ether / Dot1Q / ARP is at 18:b4:30:0d:87:18 says 192.168.1.235 / Padding
Ether / 802.1q 5c:aa:fd:4c:92:86 > ff:ff:ff:ff:ff:ff (0x6970) vlan 1 / Raw
Ether / Dot1Q / ARP who has 192.168.1.219 says 192.168.1.233 / Padding
Ether / Dot1Q / ARP who has 192.168.1.224 says 192.168.1.1
Ether / Dot1Q / ARP who has 192.168.1.208 says 192.168.1.1
Ether / Dot1Q / ARP who has 192.168.1.225 says 192.168.1.1
Ether / Dot1Q / ARP is at 38:c9:86:40:7c:a6 says 192.168

In [10]:
dframe = pd.DataFrame.transpose(pd.DataFrame(d))
dframe

,DNS QD,HTTP,MDNS AN,MDNS NS,Manufacturer
28:f0:76:31:d3:58,b'clients6.google.com.',NaN,NaN,NaN,Apple
40:cb:c0:bc:36:7e,NaN,NaN,b'_sleep-proxy._udp.local.',NaN,Apple
78:28:ca:03:80:0c,NaN,{'151.101.40.246': None},NaN,NaN,Sonos
84:38:35:5a:70:40,b'www.googleadservices.com.',NaN,NaN,NaN,Apple
88:15:44:aa:47:2e,b'8.246.137.98.in-addr.arpa.',NaN,NaN,NaN,CiscoMer
b4:7c:9c:31:b4:df,b'www.example.org.',"{'93.184.216.34': None, '52.216.17.168': None,...",b'Android.local.',b'Android.local.',AmazonTe
e0:55:3d:02:41:21,b'www.googleadservices.com.',NaN,b'Android.local.',NaN,CiscoMer


In [151]:
#columns = ['MAC ID', 'Manufacturer',]
#df = pd.DataFrame(columns=columns)

IPID_count          : Identify IP id values classes in a list of packets
arpcachepoison      : Poison target's cache with (your MAC,victim's IP) couple
arping              : Send ARP who-has requests to determine which hosts are up
bind_layers         : Bind 2 layers on some specific fields' values
bridge_and_sniff    : Forward traffic between interfaces if1 and if2, sniff and return
chexdump            :  Build a per byte hexadecimal representation
computeNIGroupAddr  : Compute the NI group Address. Can take a FQDN as input parameter
corrupt_bits        : Flip a given percentage or number of bits from a string
corrupt_bytes       : Corrupt a given percentage or number of bytes from a string
defrag              : defrag(plist) -> ([not fragmented], [defragmented],
defragment          : defrag(plist) -> plist defragmented as much as possible 
dhcp_request        : --
dyndns_add          : Send a DNS add message to a nameserver for "name" to have a new "rdata"
dyndns_del          : Send 

In [59]:
import pycurl
from io import StringIO

c = pycurl.Curl()
c.setopt(c.URL, 'https://ipinfo.io/216.58.195.243?token=4b22562280ac95')

e = io.BytesIO()
c.setopt(pycurl.WRITEFUNCTION, e.write)
c.perform()
c.close()
htmlString = e.getvalue().decode('UTF-8')
print(htmlString)

{
  "ip": "216.58.195.243",
  "hostname": "sfo03s06-in-f19.1e100.net",
  "city": "Mountain View",
  "region": "California",
  "country": "US",
  "loc": "37.4192,-122.0570",
  "postal": "94043",
  "org": "AS15169 Google LLC"
}


In [60]:
type(htmlString)

str